In [48]:
import pandas as pd
from pathlib import Path
import numpy as np

In [49]:
data_folder = Path('../../data')
fn_feed = data_folder /'this_project/3_keio_strains_chemostat/3B_chemostat_feed_rates.csv'
df_feed = pd.read_csv(fn_feed)

In [51]:
df_feed.loc[df_feed['Time [h]'].isna(), :]

,Time [h],aceE-M2,aceE-M3,aceE-M4,sucB-M5,sucB-M6,sucB-M7


In [52]:
strains = df_feed.columns[1:]
generations_dict = {}
volume = 300 #mL
for strain in strains:
    prev_time = 0
    generations = 0

    for i, row in df_feed.iterrows():
        if np.isnan(row[strain]):
            prev_time = row['Time [h]']
            continue
        if row[strain]>0:
            dilution_rate = row[strain]/volume
            doubling_time = np.log(2)/dilution_rate
            generations += (row['Time [h]'] - prev_time)/doubling_time
            if np.isnan(generations):
                print(f'{strain} {row["Time [h]"]} {prev_time} {dilution_rate} {doubling_time} {generations}')
        prev_time = row['Time [h]']
        
    generations_dict[strain] = generations
             


        

# Add generations from batch mode

In [56]:
fn_od = data_folder /'this_project/3_keio_strains_chemostat/3A_chemostat_OD.csv'
df_od = pd.read_csv(fn_od)

In [59]:
time_od_for_chemostat_start = {
    'aceE-M2': (pd.to_datetime('03.04.2024  23:00:00'), 0.69),
    'aceE-M3': (pd.to_datetime('03.04.2024  23:00:00'), 1.04),
    'aceE-M4': (pd.to_datetime('05.04.2024  16:30:00'), 0.73),
    'sucB-M5': (pd.to_datetime('03.04.2024  08:30:00'), 3.1),
    'sucB-M6': (pd.to_datetime('03.04.2024  08:30:00'), 3),
    'sucB-M7': (pd.to_datetime('03.04.2024  08:30:00'),3.3),
}

In [60]:
inoculation_od = 0.1
for strain, (time, od) in time_od_for_chemostat_start.items():
    batch_generations = (np.log(od)-np.log(inoculation_od))/np.log(2)
    generations_dict[strain] += batch_generations
    print(f'{strain} {batch_generations} {generations_dict[strain]}')

aceE-M2 2.786596361890806 109.35230632231377
aceE-M3 3.3785116232537296 120.57190932103755
aceE-M4 2.8678964639926545 121.92053230799293
sucB-M5 4.954196310386876 224.76890970729374
sucB-M6 4.906890595608518 225.98192941304748
sucB-M7 5.044394119358453 200.03003328433502


In [61]:
for strain, generations in generations_dict.items():
    print(f'{strain} {generations}')

aceE-M2 109.35230632231377
aceE-M3 120.57190932103755
aceE-M4 121.92053230799293
sucB-M5 224.76890970729374
sucB-M6 225.98192941304748
sucB-M7 200.03003328433502
